# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [3]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,31.51,93,0,5.75,clear sky
1,1,Barrow,US,71.2906,-156.7887,-2.18,84,20,13.80,mist
2,2,Tukrah,LY,32.5341,20.5791,63.05,78,6,5.84,light rain
3,3,Yellowknife,CA,62.4560,-114.3525,25.32,100,100,12.66,light snow
4,4,Kommunisticheskiy,RU,61.6789,64.4819,1.24,89,100,11.36,overcast clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,69.55,85,54,1.99,broken clouds
6,6,Goedereede,NL,51.8175,3.9806,52.05,91,100,4.00,overcast clouds
7,7,Vaini,TO,-21.2000,-175.2000,77.16,94,40,9.22,drizzle
8,8,Roura,GF,4.7263,-52.3274,76.91,94,20,2.30,few clouds
9,9,Hasaki,JP,35.7333,140.8333,56.21,80,100,12.73,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]

                                    
# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Vaini,TO,-21.2000,-175.2000,77.16,94,40,9.22,drizzle
8,8,Roura,GF,4.7263,-52.3274,76.91,94,20,2.30,few clouds
11,11,Sambava,MG,-14.2667,50.1667,78.12,67,6,4.85,clear sky
15,15,Vila Velha,BR,-20.3297,-40.2925,76.95,89,75,11.50,broken clouds
24,24,Ternate,ID,0.8000,127.4000,85.37,68,41,6.11,scattered clouds
33,33,Ouricuri,BR,-7.8825,-40.0817,79.29,57,79,4.32,broken clouds
34,34,Bambous Virieux,MU,-20.3428,57.7575,78.22,47,40,10.36,scattered clouds
48,48,Kokopo,PG,-4.3520,152.2633,84.72,61,76,4.74,broken clouds
51,51,Sinnamary,GF,5.3833,-52.9500,78.13,86,11,10.38,few clouds
58,58,Butaritari,KI,3.0707,172.7902,80.89,80,30,9.60,scattered clouds


In [6]:
preferred_cities_df.count()

Unnamed: 0             143
City                   143
Country                143
Lat                    143
Lng                    143
Max Temp               143
Humidity               143
Cloudiness             143
Wind Speed             143
Current Description    143
dtype: int64

In [7]:
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [8]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Vaini,TO,-21.2000,-175.2000,77.16,94,40,9.22,drizzle
8,8,Roura,GF,4.7263,-52.3274,76.91,94,20,2.30,few clouds
11,11,Sambava,MG,-14.2667,50.1667,78.12,67,6,4.85,clear sky
15,15,Vila Velha,BR,-20.3297,-40.2925,76.95,89,75,11.50,broken clouds
24,24,Ternate,ID,0.8000,127.4000,85.37,68,41,6.11,scattered clouds
33,33,Ouricuri,BR,-7.8825,-40.0817,79.29,57,79,4.32,broken clouds
34,34,Bambous Virieux,MU,-20.3428,57.7575,78.22,47,40,10.36,scattered clouds
48,48,Kokopo,PG,-4.3520,152.2633,84.72,61,76,4.74,broken clouds
51,51,Sinnamary,GF,5.3833,-52.9500,78.13,86,11,10.38,few clouds
58,58,Butaritari,KI,3.0707,172.7902,80.89,80,30,9.60,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [9]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description
7,Vaini,TO,77.16,-21.2000,-175.2000,drizzle
8,Roura,GF,76.91,4.7263,-52.3274,few clouds
11,Sambava,MG,78.12,-14.2667,50.1667,clear sky
15,Vila Velha,BR,76.95,-20.3297,-40.2925,broken clouds
24,Ternate,ID,85.37,0.8000,127.4000,scattered clouds
33,Ouricuri,BR,79.29,-7.8825,-40.0817,broken clouds
34,Bambous Virieux,MU,78.22,-20.3428,57.7575,scattered clouds
48,Kokopo,PG,84.72,-4.3520,152.2633,broken clouds
51,Sinnamary,GF,78.13,5.3833,-52.9500,few clouds
58,Butaritari,KI,80.89,3.0707,172.7902,scattered clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [10]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
7,Vaini,TO,77.16,-21.2000,-175.2000,drizzle,
8,Roura,GF,76.91,4.7263,-52.3274,few clouds,
11,Sambava,MG,78.12,-14.2667,50.1667,clear sky,
15,Vila Velha,BR,76.95,-20.3297,-40.2925,broken clouds,
24,Ternate,ID,85.37,0.8000,127.4000,scattered clouds,
33,Ouricuri,BR,79.29,-7.8825,-40.0817,broken clouds,
34,Bambous Virieux,MU,78.22,-20.3428,57.7575,scattered clouds,
48,Kokopo,PG,84.72,-4.3520,152.2633,broken clouds,
51,Sinnamary,GF,78.13,5.3833,-52.9500,few clouds,
58,Butaritari,KI,80.89,3.0707,172.7902,scattered clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [11]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [12]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
7,Vaini,TO,77.16,-21.2000,-175.2000,drizzle,Keleti Beach Resort
8,Roura,GF,76.91,4.7263,-52.3274,few clouds,Amazonie Découverte SARL
11,Sambava,MG,78.12,-14.2667,50.1667,clear sky,Melrose
15,Vila Velha,BR,76.95,-20.3297,-40.2925,broken clouds,Hotel Vitória Palace
24,Ternate,ID,85.37,0.8000,127.4000,scattered clouds,Sahid Bela Ternate Hotel
33,Ouricuri,BR,79.29,-7.8825,-40.0817,broken clouds,Sertão Pousada
34,Bambous Virieux,MU,78.22,-20.3428,57.7575,scattered clouds,Casa Tia Villa
48,Kokopo,PG,84.72,-4.3520,152.2633,broken clouds,Kokopo Beach Bungalow Resort
51,Sinnamary,GF,78.13,5.3833,-52.9500,few clouds,ALETHEIA Appartement Sinnamary
58,Butaritari,KI,80.89,3.0707,172.7902,scattered clouds,Isles Sunset Lodge


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [14]:
hotel_df.count()

City                   143
Country                143
Max Temp               143
Lat                    143
Lng                    143
Current Description    143
Hotel Name             143
dtype: int64

In [15]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Lat                    0
Lng                    0
Current Description    0
Hotel Name             0
dtype: int64

In [16]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
7,Vaini,TO,77.16,-21.2000,-175.2000,drizzle,Keleti Beach Resort
8,Roura,GF,76.91,4.7263,-52.3274,few clouds,Amazonie Découverte SARL
11,Sambava,MG,78.12,-14.2667,50.1667,clear sky,Melrose
15,Vila Velha,BR,76.95,-20.3297,-40.2925,broken clouds,Hotel Vitória Palace
24,Ternate,ID,85.37,0.8000,127.4000,scattered clouds,Sahid Bela Ternate Hotel
33,Ouricuri,BR,79.29,-7.8825,-40.0817,broken clouds,Sertão Pousada
34,Bambous Virieux,MU,78.22,-20.3428,57.7575,scattered clouds,Casa Tia Villa
48,Kokopo,PG,84.72,-4.3520,152.2633,broken clouds,Kokopo Beach Bungalow Resort
51,Sinnamary,GF,78.13,5.3833,-52.9500,few clouds,ALETHEIA Appartement Sinnamary
58,Butaritari,KI,80.89,3.0707,172.7902,scattered clouds,Isles Sunset Lodge


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [17]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [18]:
# Set the file name.
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label='City ID')

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [19]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [20]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>drizzle and 77.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Amazonie Découverte SARL</dd>\n<dt>City</dt><dd>Roura</dd>\n<dt>Country</dt><dd>GF</dd>\n<dt>Current Weather</dt><dd>few clouds and 76.91 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Melrose</dd>\n<dt>City</dt><dd>Sambava</dd>\n<dt>Country</dt><dd>MG</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.12 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Vitória Palace</dd>\n<dt>City</dt><dd>Vila Velha</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>broken clouds and 76.95 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sahid Bela Ternate Hotel</dd>\n<dt>City</dt><dd>Ternate</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 85.37 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sertão Pousada</dd>\n<dt>City

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [21]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
7,-21.2000,-175.2000
8,4.7263,-52.3274
11,-14.2667,50.1667
15,-20.3297,-40.2925
24,0.8000,127.4000
33,-7.8825,-40.0817
34,-20.3428,57.7575
48,-4.3520,152.2633
51,5.3833,-52.9500
58,3.0707,172.7902


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [22]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))